# Scaling Portfolio Trades
Define portfolio holdings, share prices and index weights

In [15]:
portfolio = {
    "AGL": [9570,584.19],
    "GFI": [12207,240.2],
    "ANG": [5817,416.71],
    "IMP": [11301,171.03],
    "SOL": [7752,247],
    "BHG": [2835,547.95],
    "SSW": [38940,37.84],
    "GLN": [12300, 103.49],
    "AMS": [240,1005],
    "NPH": [751,151.33]}

index_weights = {
    "AGL": 0.2875,
    "GFI": 0.1510,
    "ANG": 0.1247,
    "IMP": 0.0995,
    "SOL": 0.0985,
    "BHG": 0.0798,
    "SSW": 0.0757,
    "GLN": 0.0654,
    "AMS": 0.0123,
    "NPH": 0.0056}

Determine the values to trade for each share to get to index

In [16]:
def portfolio_value(portfolio):
    return sum(map(lambda x: x[0] * x[1],portfolio.values()))
    

def trade_to_index(portfolio, index_weights, cash_flow):
    trade_values = index_weights.copy()
    port_val = portfolio_value(portfolio) + cash_flow
    for share_code in index_weights:
        trade_values[share_code] = port_val * index_weights[share_code] - portfolio[share_code][0]*portfolio[share_code][1]
    return trade_values

trade_to_index(portfolio, index_weights, -1100000)    

{'AGL': -315337.94950000104,
 'GFI': -161410.3985199998,
 'ANG': -135871.85884399945,
 'IMP': -107076.62173999986,
 'SOL': -107359.6712199999,
 'BHG': -89181.70749600022,
 'SSW': -84464.28336400003,
 'GLN': -72897.20200799988,
 'AMS': -15506.322396000003,
 'NPH': -10893.984912000014}

Scale the values so that it is either 0 or above minimum trading size

In [17]:
def scale_values(trade_values, cash_flow, min_trade_size):
    new_trades = trade_values.copy()
    count = 0
    redistribute = 0
    for code in trade_values:
        
        if abs(trade_values[code]) < min_trade_size:
            
            if abs(trade_values[code] * 2) > min_trade_size:
                
                if trade_values[code] < 0:
                    new_trades[code] = -min_trade_size
                    redistribute = redistribute + trade_values[code] + min_trade_size
                else:
                    new_trades[code] = min_trade_size
                    redistribute = redistribute + trade_values[code] - min_trade_size
            else:
                new_trades[code] = 0
                redistribute = redistribute + trade_values[code]
        else:
            count = count + 1
    
    print("Redistribute: ",redistribute)
    
    for code in trade_values:
        
        if new_trades[code] == trade_values[code]:
            new_trades[code] = trade_values[code] + redistribute / count
            
    return new_trades


trade_values = trade_to_index(portfolio, index_weights, -1100000)
scale_values(trade_values, -1100000, 25000)
             
    
                

Redistribute:  -1400.3073080000177


{'AGL': -315512.98791350104,
 'GFI': -161585.43693349982,
 'ANG': -136046.89725749946,
 'IMP': -107251.66015349986,
 'SOL': -107534.70963349991,
 'BHG': -89356.74590950023,
 'SSW': -84639.32177750004,
 'GLN': -73072.24042149988,
 'AMS': -25000,
 'NPH': 0}

In [18]:
def cash_from_trade(portfolio, trades):
    
    cash = 0
    
    for code in trades:
        cash = cash + portfolio[code][1] * trades[code]
    
    print("Cash generated by trades: ",-cash)
    
    return cash
        

Function that print each share's deviationand check tolerance level

In [19]:
def check_tolerance(portfolio, index_weights, trade_shares, tolerance):
    new_port = portfolio.copy()
    for code in trade_shares:
        new_port[code][0] = portfolio[code][0] + trade_shares[code]
    
    port_val = portfolio_value(new_port)
    print("CODE PRT_W IDX_W DIFF")
    for code in new_port:
        pweight = new_port[code][0] * new_port[code][1] / port_val
        print(code,round(pweight,4),index_weights[code],round(pweight - index_weights[code],4))
        if abs(pweight - index_weights[code]) > tolerance:
            print("Tolerance exceeded for",code,": Difference from index: ",round(pweight - index_weights[code],3),". Lower minimum trade value or increase tolerance.")
            return False
    print("Portfolio within tolerance.")
    
    return True
    
        

Finally determine number of (scaled) shares to trade after a tolerance check

In [20]:
def scale_trades(portfolio, index_weights, cash_flow, min_trade_size, tolerance):
    
    trade_values = trade_to_index(portfolio, index_weights, cash_flow)
    scaling_values = scale_values(trade_values, cash_flow, min_trade_size)
    trade_shares = dict.fromkeys(trade_values.keys(), 0)
    
    for code in trade_shares:
        trade_shares[code] = scaling_values[code] // portfolio[code][1]
    
    # check if new portfolio is within tolerance and return shares to be traded if so
    
    if check_tolerance(portfolio, index_weights, trade_shares, tolerance):
        cash_from_trade(portfolio, trade_shares)
        return trade_shares    
    

scale_trades(portfolio, index_weights, -1100000, 25000, 0.001)

Redistribute:  -1400.3073080000177
CODE PRT_W IDX_W DIFF
AGL 0.2875 0.2875 -0.0
GFI 0.151 0.151 0.0
ANG 0.1247 0.1247 -0.0
IMP 0.0995 0.0995 -0.0
SOL 0.0985 0.0985 -0.0
BHG 0.0798 0.0798 -0.0
SSW 0.0757 0.0757 -0.0
GLN 0.0654 0.0654 -0.0
AMS 0.0118 0.0123 -0.0005
NPH 0.0062 0.0056 0.0006
Portfolio within tolerance.
Cash generated by trades:  1101868.71


{'AGL': -541.0,
 'GFI': -673.0,
 'ANG': -327.0,
 'IMP': -628.0,
 'SOL': -436.0,
 'BHG': -164.0,
 'SSW': -2237.0,
 'GLN': -707.0,
 'AMS': -25,
 'NPH': 0.0}